## Working With Pandas

In this lecture we explore _aggregating_ data loaded into Pandas DataFrames and Series.

Aggregating means applying functions (such as `sum()`, `mean()` or `median()`) that take a dataset and return a summary value (e.g., a statistic)

![](figures/03.08-split-apply-combine.png)

A magic `display()` function that renders results of expressions side by side.

In [1]:
import numpy as np
import pandas as pd

class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)

## The Extrasolar Planet dataset

For this lecture, we'll use the Extrasolar Planets dataset that comes included with [Seaborn](http://seaborn.pydata.org/) (a visualization package). This dataset contains the basic information about exoplanet discoveries.

In [3]:
import seaborn as sns
planets = sns.load_dataset('planets')
planets.shape

(1035, 6)

In [6]:
planets.head(200)

,method,number,orbital_period,mass,distance,year
0,Radial Velocity,1,269.300000,7.100,77.40,2006
1,Radial Velocity,1,874.774000,2.210,56.95,2008
2,Radial Velocity,1,763.000000,2.600,19.84,2011
3,Radial Velocity,1,326.030000,19.400,110.62,2007
4,Radial Velocity,1,516.220000,10.500,119.47,2009
5,Radial Velocity,1,185.840000,4.800,76.39,2008
6,Radial Velocity,1,1773.400000,4.640,18.15,2002
7,Radial Velocity,1,798.500000,NaN,21.41,1996
8,Radial Velocity,1,993.300000,10.300,73.10,2008
9,Radial Velocity,2,452.800000,1.990,74.79,2010


## Simple Aggregation in Pandas

Aggregating data in Pandas series is analogous to aggregating data in `numpy` arrays:

In [14]:
rng = np.random.RandomState(42)
ser = pd.Series(rng.rand(5))
ser

0    0.374540
1    0.950714
2    0.731994
3    0.598658
4    0.156019
dtype: float64

In [15]:
ser.sum()

2.811925491708157

In [16]:
ser.mean()

0.5623850983416314

If you do the same with a DataFrame, it will aggregate results on a per-column basis:

In [17]:
df = pd.DataFrame({'A': rng.rand(5),
                   'B': rng.rand(5)})
df

,A,B
0,0.155995,0.020584
1,0.058084,0.969910
2,0.866176,0.832443
3,0.601115,0.212339
4,0.708073,0.181825


In [18]:
df.mean()

A    0.477888
B    0.443420
dtype: float64

You can switch to per-row using the `axis` argument:

In [21]:
df.mean(axis='columns')

0    0.088290
1    0.513997
2    0.849309
3    0.406727
4    0.444949
dtype: float64

A convenience method `.describe` can be very useful to get a quick handle on a dataset:
- note: `.dropna` removes missing values

In [12]:
#planets.describe()
planets.dropna().describe()

,number,orbital_period,mass,distance,year
count,498.00000,498.000000,498.000000,498.000000,498.000000
mean,1.73494,835.778671,2.509320,52.068213,2007.377510
std,1.17572,1469.128259,3.636274,46.596041,4.167284
min,1.00000,1.328300,0.003600,1.350000,1989.000000
25%,1.00000,38.272250,0.212500,24.497500,2005.000000
50%,1.00000,357.000000,1.245000,39.940000,2009.000000
75%,2.00000,999.600000,2.867500,59.332500,2011.000000
max,6.00000,17337.500000,25.000000,354.000000,2014.000000


From above: 
* Half of all known planets were discovered after 2010; this is the Kepler mission coming on line!

## Split, Apply, Combine

Simple aggregations are useful, but insufficient for real data analysis work: we would typically like to perform the aggregation on _subsets_ that are grouped by on some common property.

This is where the `groupby` function comes in: it allows us to reorganize dataset into _groups_ and apply aggregates onto each of those groups.

This is also know as the *split, apply, combine* technique.

### Split, apply, combine


![](figures/03.08-split-apply-combine.png)

Let's explore these capabilities by creating the `DataFrame` from the example above:

In [14]:
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'data': range(6)}, columns=['key', 'data'])
df

,key,data
0,A,0
1,B,1
2,C,2
3,A,3
4,B,4
5,C,5


Group by values of a column named `key`:

In [15]:
df.groupby('key')

A `DataFrameGroupBy` object is returned. It allows Pandas to efficiently compute the statistics, without actually physically materializing the split `DataFrames`.

To actually produce a result, we must apply the desired aggregate function:

In [16]:
df.groupby('key').sum()

,data
key,
A,3
B,5
C,7


`sum()` is just an example here; you can apply any of the Pandas or Numpy aggregate methods!

## Question: what would `df.groupby('data').sum()` do?

### The `*GroupBy` object

The `*GroupBy` object behaves in many way as a regular DataFrame.

For example, you can still select individual columns:

In [26]:
planets.groupby('method')

In [27]:
planets.groupby('method')['orbital_period']

Computation is performed once we invoke an aggregate function:

In [30]:
planets.groupby('method')['orbital_period'].median()

method
Astrometry                         631.180000
Eclipse Timing Variations         4343.500000
Imaging                          27500.000000
Microlensing                      3300.000000
Orbital Brightness Modulation        0.342887
Pulsar Timing                       66.541900
Pulsation Timing Variations       1170.000000
Radial Velocity                    360.200000
Transit                              5.714932
Transit Timing Variations           57.011000
Name: orbital_period, dtype: float64

N.b.: this shows us the typical orbital period that each method is sensitive to.

## Lazy Evaluation

When `*GroupBy` objects are constructed, nothing is actually computed. Only an object is created that lets Pandas know how to construct groups once an aggregation is actually requested (e.g., by invoking `.median()` above).

This is what's known as _lazy evaluation_ -- nothing is computed until the result is actually needed. It allows Pandas to internally optimize these expressions, once all the details of what the user wants are known.

## Dispatch methods

All methods you could call on the DataFrame or Series can be called on the `*GroupBy` objects (this is called _dispatching_).

For example, you can call `.describe()`:

In [45]:
planets.groupby('method')['year'].describe()

,count,mean,std,min,25%,50%,75%,max
method,,,,,,,,
Astrometry,2.0,2011.500000,2.121320,2010.0,2010.75,2011.5,2012.25,2013.0
Eclipse Timing Variations,9.0,2010.000000,1.414214,2008.0,2009.00,2010.0,2011.00,2012.0
Imaging,38.0,2009.131579,2.781901,2004.0,2008.00,2009.0,2011.00,2013.0
Microlensing,23.0,2009.782609,2.859697,2004.0,2008.00,2010.0,2012.00,2013.0
Orbital Brightness Modulation,3.0,2011.666667,1.154701,2011.0,2011.00,2011.0,2012.00,2013.0
Pulsar Timing,5.0,1998.400000,8.384510,1992.0,1992.00,1994.0,2003.00,2011.0
Pulsation Timing Variations,1.0,2007.000000,NaN,2007.0,2007.00,2007.0,2007.00,2007.0
Radial Velocity,553.0,2007.518987,4.249052,1989.0,2005.00,2009.0,2011.00,2014.0
Transit,397.0,2011.236776,2.077867,2002.0,2010.00,2012.0,2013.00,2014.0


### Aggregate, filter, transform, apply

A few more functions that are frequently useful:

| Function                 | Description                                     |
|--------------------------|-------------------------------------------------|
| ``aggregate()``          | Apply custom function to create an aggregate    |
| ``filter()``             | Filter on some aspect of the DF                 |
| ``transform()``          | Transform the dataset based on group properties |
| ``apply()``              | Apply an arbitrary function to the result       |

Create a `DataFrame` for the next few demos:

In [34]:
rng = np.random.RandomState(0)
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'data1': range(6),
                   'data2': rng.randint(0, 10, 6)},
                   columns = ['key', 'data1', 'data2'])
df

,key,data1,data2
0,A,0,5
1,B,1,0
2,C,2,3
3,A,3,3
4,B,4,7
5,C,5,9


#### `aggregate()`

Aggregate allows us to call multiple aggregate functions together, including custom ones. It can take strings, function objects, dictionaries, etc. It applies these functions to the results of the groupby. Examples:

In [45]:
df.groupby('key').aggregate(['min', np.median, np.mean, max])

data1                 data2                
      min median mean max   min median mean max
key                                            
A       0    1.5  1.5   3     3    4.0  4.0   5
B       1    2.5  2.5   4     0    3.5  3.5   7
C       2    3.5  3.5   5     3    6.0  6.0   9

Dictionary, mapping columns to aggregates (apply functions on a named column basis) :

In [46]:
df.groupby('key').aggregate({'data1': min,
                             'data2': 'max'})

,data1,data2
key,,
A,0,5
B,1,7
C,2,9


#### `filter()`

Filtering lets you drop subsets of the ***original dataset*** based on the properties of the group.

Example: keep only groups where the standard deviation is larger than a given value:

In [47]:
def filter_func(df):
    return df['data2'].std() > 4

display('df', "df.groupby('key').std()", "df.groupby('key').filter(filter_func)")

df
  key  data1  data2
0   A      0      5
1   B      1      0
2   C      2      3
3   A      3      3
4   B      4      7
5   C      5      9

df.groupby('key').std()
       data1     data2
key                   
A    2.12132  1.414214
B    2.12132  4.949747
C    2.12132  4.242641

df.groupby('key').filter(filter_func)
  key  data1  data2
1   B      1      0
2   C      2      3
4   B      4      7
5   C      5      9

The same as above, but written using _anonymous functions_ (a.k.a. _lambda expressions_):

In [48]:
df.groupby('key').filter(lambda df: df.data2.std() > 4)

,key,data1,data2
1,B,1,0
2,C,2,3
4,B,4,7
5,C,5,9


#### `transform()`

Transform transforms the individual _columns_ (Series) of the input `DataFrame` in some group-dependent way. The shape of the output is the same as the input (modulo the groupby column being removed).

A typical use for `transform()` is to center the data on the per-group mean:

In [49]:
df.groupby('key').transform(lambda x: x - x.mean())

,data1,data2
0,-1.5,1.0
1,-1.5,-3.5
2,-1.5,-3.0
3,1.5,-1.0
4,1.5,3.5
5,1.5,3.0


#### `apply()`

The `apply()` method lets you apply an arbitrary function to the group results. `apply()` takes a `DataFrame`, and returns either a `DataFrame`, a `Series`, or a scalar. Those are then combined into the output `DataFrame`.

Here's an example normalizing the `data1` column by the sum of the `data2` column.

In [50]:
def norm_by_data2(x):
    # x is a DataFrame of group values
    x['data1'] /= x['data2'].sum()
    return x

display('df', "df.groupby('key').apply(norm_by_data2)")

,key,data1,data2
0,A,0,5
1,B,1,0
2,C,2,3
3,A,3,3
4,B,4,7
5,C,5,9
,key,data1,data2
0,A,0.000000,5
1,B,0.142857,0
2,C,0.166667,3


### Example: Exoplanet Discoveries by Method and Decade (v1)

In [51]:
decade = 10 * (planets['year'] // 10) # // is the floor
decade = decade.astype(str) + 's'
decade.name = 'decade'
planets.groupby(['method', decade])['number'].sum().unstack().fillna(0)

decade,1980s,1990s,2000s,2010s
method,,,,
Astrometry,0.0,0.0,0.0,2.0
Eclipse Timing Variations,0.0,0.0,5.0,10.0
Imaging,0.0,0.0,29.0,21.0
Microlensing,0.0,0.0,12.0,15.0
Orbital Brightness Modulation,0.0,0.0,0.0,5.0
Pulsar Timing,0.0,9.0,1.0,1.0
Pulsation Timing Variations,0.0,0.0,1.0,0.0
Radial Velocity,1.0,52.0,475.0,424.0
Transit,0.0,0.0,64.0,712.0


In [57]:
planets.groupby(['method',decade])['number'].sum()

method                         decade
Astrometry                     2010s       2
Eclipse Timing Variations      2000s       5
                               2010s      10
Imaging                        2000s      29
                               2010s      21
Microlensing                   2000s      12
                               2010s      15
Orbital Brightness Modulation  2010s       5
Pulsar Timing                  1990s       9
                               2000s       1
                               2010s       1
Pulsation Timing Variations    2000s       1
Radial Velocity                1980s       1
                               1990s      52
                               2000s     475
                               2010s     424
Transit                        2000s      64
                               2010s     712
Transit Timing Variations      2010s       9
Name: number, dtype: int64

### Example: Exoplanet Discoveries by Method and Decade (v2)

In [63]:
(
    planets
    .assign(decade = lambda df: 10 * (df.year // 10))         # Compute the decade
    .assign(decade = lambda df: df.decade.astype(str) + 's')  # Convert decate to string, append 's'
    .groupby(['method', 'decade'])                            # Group by method (1st) and decade (2nd)
    ['number']                                                # Keep only the number of planets
    .sum()                                                    # Sum each group
    .unstack()                                                # Turn the 'decade' values into columns
    .fillna(0)                                                # Turn NAs into zeros
)

decade,1980s,1990s,2000s,2010s
method,,,,
Astrometry,0.0,0.0,0.0,2.0
Eclipse Timing Variations,0.0,0.0,5.0,10.0
Imaging,0.0,0.0,29.0,21.0
Microlensing,0.0,0.0,12.0,15.0
Orbital Brightness Modulation,0.0,0.0,0.0,5.0
Pulsar Timing,0.0,9.0,1.0,1.0
Pulsation Timing Variations,0.0,0.0,1.0,0.0
Radial Velocity,1.0,52.0,475.0,424.0
Transit,0.0,0.0,64.0,712.0


## Method Chaining

The above style is an example of [_method chaining_](https://tomaugspurger.github.io/method-chaining). Depending on your taste, it may be more (or less) readable than the more typical imperative style. It is becoming quite idiomatic with Pandas.

I personally like it because it mimics the construction of a "pipeline" through which the data can be imagined to "flow".

### For More

We've just skimmed the surface of what's possible with Pandas.

See Chapter 3. of [Python Data Science Handbook](http://shop.oreilly.com/product/0636920034919.do) by Jake VanderPlas; with the content also available [on GitHub](https://github.com/jakevdp/PythonDataScienceHandbook).

### For next Monday

Read through the slides on [lecture 08](https://github.com/connolly/A302_2019/blob/master/lectures/lecture-08-databases/lecture-08-astr302-databases.pdf) 

Install [sqlite](https://www.sqlite.org/download.html)